# Convert abinit ddb file to phonopy format

This script should be run in the conda environment obtained using provided `abipy_env.yml` file

Use following command : `conda env create -f abipy_env.yml`

**All the ddb files can be downloaded from [here](https://springernature.figshare.com/articles/dataset/DDB_files/5857959?backTo=%2Fcollections%2FHigh-throughput_Density-Functional_Perturbation_Theory_phonons_for_inorganic_materials%2F3938023&file=10399272)**

Before running the code snippet below, ensure you have setup `manager.yml` and `scheduler.yml` file in your `~/.abinit/abipy/` directory. Please refer the abipy documentation for more details on manager and scheduler config [here](https://abinit.github.io/abipy/workflows/taskmanager.html#). Below are minimal example configs

Example `manager.yml`

```yml
# queues
qadapters:
  - priority: 1
    queue:
       qtype: shell
       qname: localhost
    job:
       mpi_runner: "mpirun"
    limits:
        timelimit: 1:00:00   #  Time-limit for each task.
        max_cores: 2         #  Max number of cores that can be used by a single task.
    hardware:
        num_nodes: 1
        sockets_per_node: 1
        cores_per_socket: 2
        mem_per_node: 4 Gb
```

Example `scheduler.yml`

```yml
max_njobs_inqueue: 2
max_ncores_used: 8


# number of seconds to wait.
seconds: 5
```

In [ ]:
import os
import pandas as pd
import warnings 
from abipy import abilab
from tqdm.autonotebook import tqdm
warnings.filterwarnings("ignore")

In [ ]:
abilab.enable_notebook() # This line tells AbiPy we are running inside a notebook

In [ ]:
parent_dir = os.getcwd()

Here, only one ddb file is included for demonstration and converted to phonopy format 

In [ ]:
path_to_ddb_file = os.path.abspath("example_phonon_db_files/ddbs_file") # absolute path is needed for abipy to correctly read the files

In [ ]:
# Get list of file ddb files to convert
os.chdir('example_phonon_db_files/ddbs_file/')
ddbs_files= [f for f in os.listdir()  if not f.startswith('.') and not f.startswith('__')
            and not os.path.isdir(f)]

# Return to root directory
os.chdir(parent_dir)

In [ ]:
df = pd.DataFrame(index=[mpid.split('_')[0] for mpid in ddbs_files], columns=['sc_matrix'])

In [ ]:
for ddb_file in tqdm(ddbs_files):
    ddb = abilab.abiopen(f"{path_to_ddb_file}/{ddb_file}")
    mpid = ddb_file.split('_')[0]
    os.makedirs(f"example_phonon_db_files/phonopy_fc/{mpid}", exist_ok=True)
    os.chdir(f"example_phonon_db_files/phonopy_fc/{mpid}")
    # The line below will fail currently as converted data already exist in directory where we are saving the outputs. 
    # Please delete the contents of directory i.e make it empty 
    phonopy_fc = ddb.anaget_phonopy_ifc(output_dir_path='.',mpi_procs=8, workdir='.')
    df.loc[mpid, 'sc_matrix'] = ddb.guessed_ngqpt
    os.chdir(parent_dir)

In [ ]:
# save the supercell matrix used in abinit ddb to phonopy conversion (need this information for mean squared displacement extraction)
df.to_json('sc_matrix.json') 